In [1]:
from random import random, seed, randrange
from math import floor, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
import pandas as pd
from scipy.integrate import simps, trapz, cumtrapz
from scipy.ndimage.filters import gaussian_filter1d
from time import time
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 9]

In [2]:
%run -i "../../carbon_walls/gcmc.py"

In [3]:
filename = "../data/" + "y0.5rep1" + ".csv"
data_dump = pd.read_csv(filename).copy()

for i in range(data_dump.shape[0]):
    if np.isnan( data_dump.yco_sim[i]):
        yco = data_dump.yco_res[i]
        P_res = data_dump.P_res[i]*10**5 #[Pa]
        T = data_dump.T_res[i] + 273.15 #K
        fco, fme = PR_Fugacity( P_res/10**5, T, yco )
        fco = fco*10**5
        fme = fme*10**5
        
        sf = False
        mega_verbose = False

        s_box = 57.15
        N_max = 50000
        Vol = s_box**3
        kb = 1.3806*10**(7) #[Pa*A^3/K]
        Nco = 0 #floor(fco*Vol/kb/T)
        Nme = 0 #floor(fme*Vol/kb/T)
        Nc = 0
        rc = s_box
        beta = 1/T
        zz_co = beta*fco
        zz_me = beta*fme
        delta = 1
        pi_move = 0.5

        N_moves = 100
        N_equil = int( np.round( 200000/N_moves) )
        N_prod = int( np.round( 100000/N_moves) )

        t0 = time()
        rhocov,rhomev,Env,Pv,Ncov, Nmev = mc_run()
        t = time()
        
        Nco = Ncov.mean()
        Nme = Nmev.mean()
        data_dump.yco_sim[i] = Nco/(Nco + Nme)
        data_dump.P_sim[i] = Pv.mean()*10 #[bar]
        data_dump.E_sim[i] = Env.mean()
        data_dump.rhoco[i] = rhocov.mean()
        data_dump.rhome[i] = rhomev.mean()
        data_dump.yco_s[i] = (rhocov/(rhocov + rhomev)).std()
        data_dump.P_s[i] = Pv.std()*10 #[bar]
        data_dump.rhoco_s[i] = rhocov.std()
        data_dump.rhome_s[i] = rhomev.std()
        data_dump.E_s[i] = Env.std()
        data_dump.time[i] = t - t0
        print( i, data_dump.yco_sim[i],  data_dump.P_sim[i], data_dump.rhoco[i], data_dump.rhome[i], data_dump.time[i] )
        #filename = "../data_output/yco" + str(yco) + "P" + str(P) + "/run" + str(i) + ".csv"
        data_dump.to_csv( filename, index = False)

6 0.4915584278378666 68.26075399023624 0.0010020501520324298 0.0010364667267035838 258.4995496273041
7 0.48594846755503457 74.01427834144515 0.0011091670905029377 0.0011733117412207627 336.0287640094757
8 0.49158847548363604 85.13442354510936 0.0013351266338318946 0.0013808170882384128 343.9074230194092
9 0.4902503842061043 90.87006486396604 0.0014930949914177245 0.0015524813906091742 381.7047212123871
10 0.4840080937286842 103.96005056298245 0.0017744537680622406 0.0018917117177094274 441.55693531036377
11 0.4734902031382368 113.64909743673321 0.001896876111132303 0.0021092809298371477 492.28124594688416
12 0.4785326994883664 131.97225105199357 0.002269053808924075 0.002472636385601989 574.1803646087646
13 0.4896674751005105 134.25271537288796 0.002518535830845897 0.002624823609821334 609.7453334331512
14 0.48085023152950446 136.1629271246065 0.0028368655741448316 0.003062820831581199 705.7140016555786
15 0.4908875595900291 153.99638079727475 0.003170189619814989 0.0032878873020418453